# Setup

In [4]:
# Standard library imports
import os
import re
import math
import json
import random
import functools
from datetime import datetime, timedelta
from typing import Any, Callable, Dict, List, Optional, Tuple
from collections import OrderedDict

# Third-party imports
import torch
import openai
import faiss
import tenacity

# LangChain imports
from langchain.utils import mock_now
from langchain.docstore import InMemoryDocstore
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.chains import LLMChain
from langchain_core.language_models import BaseLanguageModel
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage, SystemMessage, BaseMemory, Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.output_parsers import RegexParser

# Pydantic imports
from pydantic import BaseModel, Field, ConfigDict

# Hugging Face imports
import transformers
from transformers import (AutoModelForCausalLM, AutoTokenizer, AutoConfig, pipeline, AutoModel)
from peft import PeftModel, PeftConfig
from langchain_huggingface import HuggingFacePipeline

In [1]:
# Set API Keys
from kaggle_secrets import UserSecretsClient # API Loggins
user_secrets = UserSecretsClient()

## Hugging Face
Hugging_Face_token = user_secrets.get_secret("Hugging_Face_token")

## Openai
OPENAI_API_KEY = user_secrets.get_secret("OPENAI_API_KEY")

In [2]:
# Login to Hugging Face
from huggingface_hub import login

login(Hugging_Face_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Load Model

## Load Model: GPT

In [10]:
LLM_gpt  = ChatOpenAI(model="gpt-3.5-turbo", 
                 max_tokens=1500, 
                 api_key = OPENAI_API_KEY) 

In [19]:
selected_embeddings_model = OpenAIEmbeddings(api_key = OPENAI_API_KEY)
embedding_size_selectedLLM = len(selected_embeddings_model.embed_query("This is a test."))
print(f"Embedding size: {embedding_size_selectedLLM}")

Embedding size: 1536


## Load Model: Llama

In [ ]:
# Set up Tokenizer & Model & Pipeline
model_id = "meta-llama/Llama-3.2-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model = AutoModel.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

In [ ]:
# Define LLM Pipeline
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    tokenizer=tokenizer,
    max_new_tokens = 500,  # Maximum new tokens to generate
    torch_dtype=torch.float16,
    device_map="auto",
    truncation=True,
    #temperature=0.7,  # Sampling temperature
    #top_p=0.9,        # Nucleus sampling
    #repetition_penalty=1.2,  # Penalize repetition
)

LLM = HuggingFacePipeline(pipeline=pipeline)

[**Decoding Strategies**](https://towardsdatascience.com/decoding-strategies-that-you-need-to-know-for-response-generation-ba95ee0faadc)

`do_sample=True` use Sample Decoding

`do_sample=False` use Greedy decoding

In [ ]:
# Test Model Output
def response(prompt):
     sequences = pipeline(prompt,
                          do_sample=True,   # adjust
                          top_k=10,        # adjust vocabulary size
                          )
     #print('Question: ' , prompt + '\n')
     print('response: ', sequences[0]['generated_text'][len(prompt):] + '\n')


discussion = "Should the UK rejoin the European Union?"
agent_name = "Boris Johnson" 
prompt = (
    f"Consider the following discussion:\n\n"
    f"{discussion}\n\n"
    f"As {agent_name}, on a scale of 1 to 10, how relevant is this discussion to you? "
    f"Provide a number between 1 (not relevant at all) and 10 (extremely relevant)."
    f"Output only a single number, nothing else!"
)


response(prompt)

When switching to any other LLaMA-based model, you need to replace the embedding model (OpenAIEmbeddings) with an embedding generation mechanism that works with your local LLaMA model. OpenAI provides embeddings as a service, but with LLaMA, **you need to generate embeddings using the model locally**.

In [ ]:
# Custom Llama Embeddings
class CustomLlamaEmbeddings:   # Copilot genereated
    def __init__(self, model, tokenizer, device="cuda"):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        
    def embed_documents(self, texts):
        """Generate embeddings for a single query string."""
        # Tokenize inputs
        inputs = self.tokenizer(
            texts, padding=True, truncation=True, return_tensors="pt").to(self.device)
        
        # Get hidden states from the model
        with torch.no_grad():
            outputs = self.model(**inputs, return_dict=True, output_hidden_states=True)
            
        # Use the mean pooling of the last hidden state as embeddings
        embeddings = outputs.hidden_states[-1].mean(dim=1)
        return embeddings.cpu().numpy()
        
    def embed_query(self, text):
        """Generate embeddings for a list of documents."""
        return self.embed_documents([text])[0]

selected_embeddings_model = CustomLlamaEmbeddings(model=model, tokenizer=tokenizer)

In [ ]:
config = AutoConfig.from_pretrained(model_id)
# Print the embedding size
#embedding_size_selectedLLM = config.hidden_size
#print(f"Embedding Size (hidden size): {config.hidden_size}")#

## Load Model: Tuned Llama

In [12]:
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/fork-of-finetune-hoc-llama/kaggle/working/fine-tuned-llama_hoc_Boris", 
                                          use_fast=True)

base_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [13]:
# Load the fine-tuned adapter
model = PeftModel.from_pretrained(base_model, "/kaggle/input/fork-of-finetune-hoc-llama/outputs/checkpoint-93")


In [16]:
input_text = "What is your opinion on the UK rejoining the European Union?"
inputs = tokenizer(input_text, return_tensors="pt")

outputs = model.generate(**inputs, max_length=500)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


What is your opinion on the UK rejoining the European Union? to share your thoughts on the Brexit debate.
I'm happy to engage in a respectful discussion on the topic, but I want to clarify that I'll be presenting a neutral perspective, and my goal is to facilitate a constructive conversation.
The UK's decision to leave the European Union (Brexit) was a contentious issue, and opinions on it vary widely. Here's a summary of the main arguments for and against the UK rejoining the EU:

**Arguments For Rejoining the EU:**

1. **Economic benefits**: Many argue that rejoining the EU would provide the UK with access to a large, integrated market, potentially boosting trade, investment, and economic growth.
2. **Regulatory alignment**: EU membership would allow the UK to align its regulatory frameworks with those of the EU, reducing the complexity and costs associated with implementing and enforcing EU laws.
3. **Cultural and social ties**: The UK's historical and cultural ties to the EU are st

## Load Model: Tuned Boris

In [ ]:
# base_model_name = "meta-llama/Llama-3.2-3B"
PEFT_MODEL = "/kaggle/input/llama_boris/pytorch/default/1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                      # Load model in 4bit, to redeuce memory and computational requirements
    bnb_4bit_use_double_quant=True,         # Double quantization, further compress the model weights
    bnb_4bit_quant_type="nf4",              # Quantization type = nf4
    bnb_4bit_compute_dtype=torch.bfloat16,  # Compute in 16bit format, to speed up computation
    load_in_8bit_fp32_cpu_offload=True
)

config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)
tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
# Test the Fine-tuned model
prompt = "Should the UK rejoin the European Union?"
# Tokenize the input prompt
inputs = tokenizer(prompt, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")

# Generate a response
output = model.generate(
    input_ids=inputs["input_ids"],
    attention_mask=inputs["attention_mask"],  # Explicitly set the attention mask
    max_length=300,              # Maximum length of the generated response
    temperature=0.7,             # Sampling temperature for more creative responses
    top_p=0.9,                   # Nucleus sampling for generating diverse text
    repetition_penalty=1.2,      # Penalize repetition in the response
    do_sample=True,              # Enable sampling for non-deterministic output
    pad_token_id=tokenizer.eos_token_id,      # Explicitly set the pad token ID
)
# Decode and print the response
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)

In [ ]:
class CustomLlamaEmbeddings:
    def __init__(self, model, tokenizer, device="cuda"):
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        
    def embed_documents(self, texts):
        # Tokenize inputs
        inputs = self.tokenizer(
            texts, padding=True, truncation=True, return_tensors="pt").to(self.device)
        
        # Get hidden states from the model
        with torch.no_grad():
            outputs = self.model(**inputs, return_dict=True, output_hidden_states=True)
            
        # Use the mean pooling of the last hidden state as embeddings
        embeddings = outputs.hidden_states[-1].mean(dim=1)
        return embeddings.cpu().numpy()
        
    def embed_query(self, text):
        """Generate embeddings for a list of documents."""
        return self.embed_documents([text])[0]


In [ ]:
# Define LLM and Embeddings
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=500,  # Maximum tokens in the output
    temperature=0.7,  # Sampling temperature
    top_p=0.9,        # Nucleus sampling
    repetition_penalty=1.2,  # Penalize repetition
)
LLM = HuggingFacePipeline(pipeline=pipe)
selected_embeddings_model = CustomLlamaEmbeddings(model, tokenizer)


In [ ]:
embeddings = selected_embeddings_model.embed_documents(["Sample document"])
print(f"Embedding dimension: {embeddings.shape[1]}")  # Check the embedding size

# Generative AI Setup
The [codes](https://python.langchain.com/api_reference/experimental/generative_agents.html) for the classes `GenerativeAgentMemory` and `GenerativeAgent` was entirely reused from the **[LangChain Experimental](https://pypi.org/project/langchain-experimental/)** project in the LangChain Python API reference, with a few minor tweaks and proper configuration of the prompts.


## Generative Agent Memory

In [12]:
class GenerativeAgentMemory(BaseMemory):
    """Memory for the generative agent."""
    
    llm: BaseLanguageModel
    """The core language model."""
    
    memory_retriever: TimeWeightedVectorStoreRetriever
    """The retriever to fetch related memories."""
    
    verbose: bool = False
    reflection_threshold: Optional[float] = None
    """When aggregate_importance exceeds reflection_threshold, stop to reflect."""
    
    current_plan: List[str] = []
    """The current plan of the agent."""
    
    # A weight of 0.15 makes this less important than it
    # would be otherwise, relative to salience and time
    importance_weight: float = 0.15
    """How much weight to assign the memory importance."""
    aggregate_importance: float = 0.0  # : :meta private:
    """Track the sum of the 'importance' of recent memories.
    Triggers reflection when it reaches reflection_threshold."""
    max_tokens_limit: int = 1200  # : :meta private:
    
    # input keys
    queries_key: str = "queries"
    most_recent_memories_token_key: str = "recent_memories_token"
    add_memory_key: str = "add_memory"
    
    # output keys
    relevant_memories_key: str = "relevant_memories"
    relevant_memories_simple_key: str = "relevant_memories_simple"
    most_recent_memories_key: str = "most_recent_memories"
    now_key: str = "now"
    reflecting: bool = False
    
    def chain(self, prompt: PromptTemplate) -> LLMChain:
        return LLMChain(llm=self.llm, prompt=prompt, verbose=self.verbose)
    @staticmethod
    
    def _parse_list(text: str) -> List[str]:
        """Parse a newline-separated string into a list of strings."""
        lines = re.split(r"\n", text.strip())
        lines = [line for line in lines if line.strip()]  # remove empty lines
        return [re.sub(r"^\s*\d+\.\s*", "", line).strip() for line in lines]
    
    def _get_topics_of_reflection(self, last_k: int = 50) -> List[str]:
        """Return the 3 most salient high-level questions about recent observations."""
        prompt = PromptTemplate.from_template(
            "{observations}\n\n"
            "Given only the information above, what are the 3 most salient "
            "high-level questions we can answer about the subjects in the statements?\n"
            "Provide each question on a new line."
        )
        observations = self.memory_retriever.memory_stream[-last_k:]
        observation_str = "\n".join(
            [self._format_memory_detail(o) for o in observations]
        )
        result = self.chain(prompt).run(observations=observation_str)
        return self._parse_list(result)
    
    def _get_insights_on_topic(
        self, topic: str, now: Optional[datetime] = None
    ) -> List[str]:
        """Generate 'insights' on a topic of reflection, based on pertinent memories."""
        prompt = PromptTemplate.from_template(
            "Statements relevant to: '{topic}'\n"
            "---\n"
            "{related_statements}\n"
            "---\n"
            "What 5 high-level novel insights can you infer from the above statements "
            "that are relevant for answering the following question?\n"
            "Do not include any insights that are not relevant to the question.\n"
            "Do not repeat any insights that have already been made.\n\n"
            "Question: {topic}\n\n"
            "(example format: insight (because of 1, 5, 3))\n"
        )
        related_memories = self.fetch_memories(topic, now=now)
        related_statements = "\n".join(
            [
                self._format_memory_detail(memory, prefix=f"{i+1}. ")
                for i, memory in enumerate(related_memories)
            ]
        )
        result = self.chain(prompt).run(
            topic=topic, related_statements=related_statements
        )
        # TODO: Parse the connections between memories and insights
        return self._parse_list(result)
    
    def pause_to_reflect(self, now: Optional[datetime] = None) -> List[str]:
        """Reflect on recent observations and generate 'insights'."""
        if self.verbose:
            logger.info("Character is reflecting")
        new_insights = []
        topics = self._get_topics_of_reflection()
        for topic in topics:
            insights = self._get_insights_on_topic(topic, now=now)
            for insight in insights:
                self.add_memory(insight, now=now)
            new_insights.extend(insights)
        return new_insights
    
    def _score_memory_importance(self, memory_content: str) -> float:
        """Score the absolute importance of the given memory."""
        prompt = PromptTemplate.from_template(
            "On the scale of 1 to 10, where 1 is purely mundane"
            + " (e.g., brushing teeth, making bed) and 10 is"
            + " extremely poignant (e.g., a break up, college"
            + " acceptance), rate the likely poignancy of the"
            + " following piece of memory. Respond with a single integer."
            + "\nMemory: {memory_content}"
            + "\nRating: "
        )
        score = self.chain(prompt).run(memory_content=memory_content).strip()
        if self.verbose:
            logger.info(f"Importance score: {score}")
        match = re.search(r"^\D*(\d+)", score)
        if match:
            return (float(match.group(1)) / 10) * self.importance_weight
        else:
            return 0.0
    
    def _score_memories_importance(self, memory_content: str) -> List[float]:
        """Score the absolute importance of the given memory."""
        prompt = PromptTemplate.from_template(
            "On the scale of 1 to 10, where 1 is purely mundane"
            + " (e.g., brushing teeth, making bed) and 10 is"
            + " extremely poignant (e.g., a break up, college"
            + " acceptance), rate the likely poignancy of the"
            + " following piece of memory. Always answer with only a list of numbers."
            + " If just given one memory still respond in a list."
            + " Memories are separated by semi colans (;)"
            + "\nMemories: {memory_content}"
            + "\nRating: "
        )
        scores = self.chain(prompt).run(memory_content=memory_content).strip()
        if self.verbose:
            logger.info(f"Importance scores: {scores}")
        # Split into list of strings and convert to floats
        scores_list = [float(x) for x in scores.split(";")]
        return scores_list
    
    def add_memories(
        self, memory_content: str, now: Optional[datetime] = None
    ) -> List[str]:
        """Add an observations or memories to the agent's memory."""
        importance_scores = self._score_memories_importance(memory_content)
        self.aggregate_importance += max(importance_scores)
        memory_list = memory_content.split(";")
        documents = []
        for i in range(len(memory_list)):
            documents.append(
                Document(
                    page_content=memory_list[i],
                    metadata={"importance": importance_scores[i]},
                )
            )
        result = self.memory_retriever.add_documents(documents, current_time=now)
        # After an agent has processed a certain amount of memories (as measured by
        # aggregate importance), it is time to reflect on recent events to add
        # more synthesized memories to the agent's memory stream.
        if (
            self.reflection_threshold is not None
            and self.aggregate_importance > self.reflection_threshold
            and not self.reflecting
        ):
            self.reflecting = True
            self.pause_to_reflect(now=now)
            # Hack to clear the importance from reflection
            self.aggregate_importance = 0.0
            self.reflecting = False
        return result
    
    def add_memory(
        self, memory_content: str, now: Optional[datetime] = None
    ) -> List[str]:
        """Add an observation or memory to the agent's memory."""
        importance_score = self._score_memory_importance(memory_content)
        self.aggregate_importance += importance_score
        document = Document(
            page_content=memory_content, metadata={"importance": importance_score}
        )
        result = self.memory_retriever.add_documents([document], current_time=now)
        # After an agent has processed a certain amount of memories (as measured by
        # aggregate importance), it is time to reflect on recent events to add
        # more synthesized memories to the agent's memory stream.
        if (
            self.reflection_threshold is not None
            and self.aggregate_importance > self.reflection_threshold
            and not self.reflecting
        ):
            self.reflecting = True
            self.pause_to_reflect(now=now)
            # Hack to clear the importance from reflection
            self.aggregate_importance = 0.0
            self.reflecting = False
        return result
    
    def fetch_memories(
        self, observation: str, now: Optional[datetime] = None
    ) -> List[Document]:
        """Fetch related memories."""
        if now is not None:
            with mock_now(now):
                return self.memory_retriever.invoke(observation)
        else:
            return self.memory_retriever.invoke(observation)
    
    def format_memories_detail(self, relevant_memories: List[Document]) -> str:
        content = []
        for mem in relevant_memories:
            content.append(self._format_memory_detail(mem, prefix="- "))
        return "\n".join([f"{mem}" for mem in content])
    
    def _format_memory_detail(self, memory: Document, prefix: str = "") -> str:
        created_time = memory.metadata["created_at"].strftime("%B %d, %Y, %I:%M %p")
        return f"{prefix}[{created_time}] {memory.page_content.strip()}"
    
    def format_memories_simple(self, relevant_memories: List[Document]) -> str:
        return "; ".join([f"{mem.page_content}" for mem in relevant_memories])
    
    def _get_memories_until_limit(self, consumed_tokens: int) -> str:
        """Reduce the number of tokens in the documents."""
        result = []
        for doc in self.memory_retriever.memory_stream[::-1]:
            if consumed_tokens >= self.max_tokens_limit:
                break
            consumed_tokens += self.llm.get_num_tokens(doc.page_content)
            if consumed_tokens < self.max_tokens_limit:
                result.append(doc)
        return self.format_memories_simple(result)
    @property
    
    def memory_variables(self) -> List[str]:
        """Input keys this memory class will load dynamically."""
        return []
   
    def load_memory_variables(self, inputs: Dict[str, Any]) -> Dict[str, str]:
        """Return key-value pairs given the text input to the chain."""
        queries = inputs.get(self.queries_key)
        now = inputs.get(self.now_key)
        if queries is not None:
            relevant_memories = [
                mem for query in queries for mem in self.fetch_memories(query, now=now)
            ]
            return {
                self.relevant_memories_key: self.format_memories_detail(
                    relevant_memories
                ),
                self.relevant_memories_simple_key: self.format_memories_simple(
                    relevant_memories
                ),
            }
        most_recent_memories_token = inputs.get(self.most_recent_memories_token_key)
        if most_recent_memories_token is not None:
            return {
                self.most_recent_memories_key: self._get_memories_until_limit(
                    most_recent_memories_token
                )
            }
        return {}
    
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, Any]) -> None:
        """Save the context of this model run to memory."""
        # TODO: fix the save memory key
        mem = outputs.get(self.add_memory_key)
        now = outputs.get(self.now_key)
        if mem:
            self.add_memory(mem, now=now)
    
    def clear(self) -> None:
        """Clear memory contents."""
        # TODO

## Generative Agent

In [13]:
class GenerativeAgent(BaseModel):
    """Agent as a character with memory and innate characteristics."""
    name: str
    """The character's name."""
    age: Optional[int] = None
    """The optional age of the character."""
    traits: str = "N/A"
    """Permanent traits to ascribe to the character."""
    status: str
    """The traits of the character you wish not to change."""
    memory: GenerativeAgentMemory
    """The memory object that combines relevance, recency, and 'importance'."""
    llm: BaseLanguageModel
    """The underlying language model."""
    verbose: bool = False
    summary: str = ""  #: :meta private:
    """Stateful self-summary generated via reflection on the character's memory."""
    summary_refresh_seconds: int = 3600  #: :meta private:
    """How frequently to re-generate the summary."""
    last_refreshed: datetime = Field(default_factory=datetime.now)  # : :meta private:
    """The last time the character's summary was regenerated."""
    daily_summaries: List[str] = Field(default_factory=list)  # : :meta private:
    """Summary of the events in the plan that the agent took."""
    model_config = ConfigDict(
        arbitrary_types_allowed=True,
    )
    # LLM-related methods
    @staticmethod
    
    def _parse_list(text: str) -> List[str]:
        """Parse a newline-separated string into a list of strings."""
        lines = re.split(r"\n", text.strip())
        return [re.sub(r"^\s*\d+\.\s*", "", line).strip() for line in lines]
        
    def chain(self, prompt: PromptTemplate) -> LLMChain:
        """Create a chain with the same settings as the agent."""
        return LLMChain(
            llm=self.llm, prompt=prompt, verbose=self.verbose, memory=self.memory
        )
        
    def _get_entity_from_observation(self, observation: str) -> str:
        prompt = PromptTemplate.from_template(
            "What is the observed entity in the following observation? {observation}"
            + "\nEntity="
        )
        return self.chain(prompt).run(observation=observation).strip()
        
    def _get_entity_action(self, observation: str, entity_name: str) -> str:
        prompt = PromptTemplate.from_template(
            "What is the {entity} doing in the following observation? {observation}"
            + "\nThe {entity} is"
        )
        return (
            self.chain(prompt).run(entity=entity_name, observation=observation).strip()
        )

## Summarize Most relevant memories
    def summarize_related_memories(self, observation: str) -> str:
        """Summarize memories that are most relevant to an observation."""
        prompt = PromptTemplate.from_template(
            """
            {q1}?
            Context from memory:
            {relevant_memories}
            Relevant context: 
            """
        )
        entity_name = self._get_entity_from_observation(observation)
        entity_action = self._get_entity_action(observation, entity_name)
        q1 = f"What is the relationship between {self.name} and {entity_name}"
        q2 = f"{entity_name} is {entity_action}"
        return self.chain(prompt=prompt).run(q1=q1, queries=[q1, q2]).strip()
        
## Generate Summary of the agent + reaction 
    def _generate_reaction(
        self, observation: str, suffix: str, now: Optional[datetime] = None
    ) -> str:
        """React to a given observation or dialogue act."""
        prompt = PromptTemplate.from_template(
            "{agent_summary_description}"
            + "\nIt is {current_time}."
            + "\n{agent_name}'s status: {agent_status}"
            + "\nSummary of relevant context from {agent_name}'s memory:"
            + "\n{relevant_memories}"
            + "\nMost recent observations: {most_recent_memories}"
            + "\nObservation: {observation}"
            + "\n\n"
            + suffix
        )
        agent_summary_description = self.get_summary(now=now)
        relevant_memories_str = self.summarize_related_memories(observation)
        current_time_str = (
            datetime.now().strftime("%B %d, %Y, %I:%M %p")
            if now is None
            else now.strftime("%B %d, %Y, %I:%M %p")
        )
        kwargs: Dict[str, Any] = dict(
            agent_summary_description=agent_summary_description,
            current_time=current_time_str,
            relevant_memories=relevant_memories_str,
            agent_name=self.name,
            observation=observation,
            agent_status=self.status,
        )
        consumed_tokens = self.llm.get_num_tokens(
            prompt.format(most_recent_memories="", **kwargs)
        )
        kwargs[self.memory.most_recent_memories_token_key] = consumed_tokens
        return self.chain(prompt=prompt).run(**kwargs).strip()
        
## Clean response
    def _clean_response(self, text: str) -> str:
        return re.sub(f"^{self.name} ", "", text.strip()).strip()
    
## Generate Dialogue response
    def generate_dialogue_response(
        self, observation: str, now: Optional[datetime] = None) -> Tuple[bool, str]:
        """React to a given observation."""
        
        call_to_action_template = (
            "You are {agent_name}, responding to the Observation.\n"
            + "Respond exactly as {agent_name} would speak, staying fully in character and addressing the observation directly"
        )
        
        # Generating response with updated prompt
        full_result = self._generate_reaction(observation, call_to_action_template, now=now)
        #result = re.findall(r'"(.*?)"', full_result)[0]
        
        response_text = self._clean_response(full_result.strip())
        self.memory.save_context(
            {},
            {
                self.memory.add_memory_key: f"{self.name} observed "
                f"{observation} and said {response_text}",
                self.memory.now_key: now,
            },
        )
        return True, f"{self.name} said {response_text}"

## Decide if the agent wants to respond to the observation
    def decide_to_respond(self, observation: str, now: Optional[datetime] = None,
                          threshold: float = 7.0) -> bool:
        """Decide whether the agent wants to respond to the observation."""

        call_to_action_template = (
            "Consider the following discussion:\n\n"
            "{observation}\n\n"
            "As {agent_name}, on a scale of 1 to 10, how relevant is this discussion to you? "
            "Provide a number between 1 (not relevant at all) and 10 (extremely relevant)."
            "Output only a number, nothing else!"
            )
        
        full_result = self._generate_reaction(observation, call_to_action_template, now=now)
        result = full_result.strip().lower()  # Normalize result to lowercase for consistent comparison
        
        try:
            relevance_score = float(result)
        except ValueError:
            logging.warning(f"Unexpected non-numeric response from agent: {result}")
            relevance_score = 0  # Default low relevance for unexpected responses

        # Save the decision context to memory
        self.memory.save_context(
            {},
            {
                self.memory.add_memory_key: f"{self.name} observed "
                f"that the relevance of the discussion '{observation}' was scored as {result}",
                self.memory.now_key: now,
            },
        )
         
        # Check if the model returned "yes" or "no"
        if relevance_score < threshold:
            return False
        elif relevance_score >= threshold:
            return True
        else:
            print(f"Unexpected response: {result}")  # For debugging purposes
            return False
    
    ######################################################
    # Agent stateful' summary methods.                   #
    # Each dialog or response prompt includes a header   #
    # summarizing the agent's self-description. This is  #
    # updated periodically through probing its memories  #
    ######################################################
    
    def _compute_agent_summary(self) -> str:
        """"""
        prompt = PromptTemplate.from_template(
            "How would you summarize {name}'s core characteristics given the"
            + " following statements:\n"
            + "{relevant_memories}"
            + "Do not embellish."
            + "\n\nSummary: "
        )
        # The agent seeks to think about their core characteristics.
        return (
            self.chain(prompt)
            .run(name=self.name, queries=[f"{self.name}'s core characteristics"])
            .strip()
        )
    
    def get_summary(
        self, force_refresh: bool = False, now: Optional[datetime] = None
    ) -> str:
        """Return a descriptive summary of the agent."""
        current_time = datetime.now() if now is None else now
        since_refresh = (current_time - self.last_refreshed).seconds
        if (
            not self.summary
            or since_refresh >= self.summary_refresh_seconds
            or force_refresh
        ):
            self.summary = self._compute_agent_summary()
            self.last_refreshed = current_time
        age = self.age if self.age is not None else "N/A"
        return (
            f"Name: {self.name} (age: {age})"
            + f"\nInnate traits: {self.traits}"
            + f"\n{self.summary}"
        )
    
    def get_full_header(
        self, force_refresh: bool = False, now: Optional[datetime] = None
    ) -> str:
        """Return a full header of the agent's status, summary, and current time."""
        now = datetime.now() if now is None else now
        summary = self.get_summary(force_refresh=force_refresh, now=now)
        current_time_str = now.strftime("%B %d, %Y, %I:%M %p")
        return (
            f"{summary}\nIt is {current_time_str}.\n{self.name}'s status: {self.status}"
        )

# Create Agent
- [GenerativeAgentMemory](https://python.langchain.com/api_reference/experimental/generative_agents/langchain_experimental.generative_agents.memory.GenerativeAgentMemory.html): **Memory** for the generative agent 
   - `llm`
   - `memory_retriever` = create_new_memory_retriever()
   - `current_plan`
   - `reflection_threshold`
   - `add_memory` add observation/memory
- [GenerativeAgent](https://python.langchain.com/api_reference/experimental/generative_agents.html): Agent as a character with **memory** and innate **characteristics**,  
   - basics like `name`, `age` and `llm`
   - `memory` object that combines relevance, recency, and ‘importance’
   - `summary` and `summary_refresh_seconds` to set how frequently to re-generate the summary
   - `summarize_related_memories`: Summarize memories that are most relevant to an observation
   - `status` fix-objectives / traits of the character you wish not to change
   - `traits` set Permanent traits to ascribe to the character 
   - `generate_dialogue_response`

In [14]:
# Relevance Score function - relevance_score_fn()
def relevance_score_fn(score: float) -> float:
    """Return a similarity score on a scale [0, 1]."""
    return 1.0 - score / math.sqrt(2)

In [21]:
# Memory Retriever function - create_new_memory_retriever()
def create_new_memory_retriever():
    """Create a new vector store retriever unique to the agent."""
    
    embeddings_model = selected_embeddings_model  
    
    # Initialize the vectorstore as empty
    embedding_size = embedding_size_selectedLLM           #use: 1536 (GPT3.5) or 3072 (Llamma)
    
    index = faiss.IndexFlatL2(embedding_size)
    vectorstore = FAISS(
        embeddings_model.embed_query,  #use: embeddings_model.embed_query OR llama_embedding_function
        index,
        InMemoryDocstore({}),  # empty Memory docstore
        {},  # index-to-document store ID mapping
        relevance_score_fn=relevance_score_fn,
    )
    
    # Time-weighted scoring mechanism
    return TimeWeightedVectorStoreRetriever(
        vectorstore=vectorstore,
        other_score_keys=["importance"],
        k=15  # retrieve up to 15 relevant memories
    )

In [22]:
# Agent Creation function - create_debate_agent()
def create_debate_agent(name, age, traits, status, 
                        #reflection_threshold, 
                        llm):
   
    memory = GenerativeAgentMemory(
        llm=llm,
        memory_retriever=create_new_memory_retriever(),
        verbose=False,
        #reflection_threshold=reflection_threshold,  # adjust as needed for reflection frequency
    )
    
    agent = GenerativeAgent(
        name=name,
        age=age,
        traits=traits,
        status=status,
        memory_retriever=create_new_memory_retriever(),
        llm=llm,
        memory=memory,
    )
    return agent

## Define Agent Traits

'Christine Jardine', 'Kit Malthouse', 'Steve Double', 'Tim Farron', 'Jo Stevens', 'Rachael Maskell'

In [51]:
# Create agents for each MP
ChristineJardine = create_debate_agent(
    name="Christine Jardine",
    age=2019-1960,  # Example age
    traits="Advocate for EU citizens' rights, empathetic communicator",
    status="Scottish Liberal Democrat politician; Opposes ending free movement; highlights contributions of EU nationals to the UK.",
    llm=LLM_gpt
)

KitMalthouse = create_debate_agent(
    name="Kit Malthouse",
    age=2019-1966,  # Example age
    traits="Firm on immigration control, prioritizes national sovereignty",
    status="British Conservative Party politician; Supports ending free movement; focuses on controlled immigration policies.",
    llm=LLM_gpt
)

SteveDouble = create_debate_agent(
    name="Steve Double",
    age=2019-1966,  # Example age
    traits="Emphasizes national interests, supportive of government policies",
    status="British Conservative Party politician; Advocates for ending free movement; reassures EU citizens are welcome under new schemes.",
    llm=LLM_gpt
)

TimFarron = create_debate_agent(
    name="Tim Farron",
    age=2019-1970,  # Example age
    traits="Pro-European, champions individual rights",
    status="British Liberal Democrats Party politician; Opposes ending free movement; stresses benefits of EU integration.",
    llm=LLM_gpt
)

JoStevens = create_debate_agent(
    name="Jo Stevens",
    age=2019-1966,  # Example age
    traits="Defender of workers' rights, critical of government policies",
    status="British Labour Party politician; Questions impact of ending free movement on labor markets and rights.",
    llm=LLM_gpt
)

RachaelMaskell = create_debate_agent(
    name="Rachael Maskell",
    age=2019-1972,  # Example age
    traits="Advocate for social justice, focuses on community welfare",
    status="British Labour and Co-operative Party politician; Concerns about social implications of ending free movement; emphasizes inclusive policies.",
    llm=LLM_gpt
)

## Define Base Memories

In [52]:
# Creat Memory objects for each agent
ChristineJardine_memory = ChristineJardine.memory
KitMalthouse_memory = KitMalthouse.memory
SteveDouble_memory = SteveDouble.memory   
TimFarron_memory = TimFarron.memory
JoStevens_memory = JoStevens.memory
RachaelMaskell_memory = RachaelMaskell.memory

In [53]:
# Base Observations 
ChristineJardine_observations = [
    "Christine Jardine is a Liberal Democrat MP representing Edinburgh West.",
    "She has expressed concerns about ending free movement, highlighting its potential negative impact on public services and the economy.",
    "Jardine emphasizes the human cost of ending free movement, noting that many EU nationals in her constituency feel unwelcome and uncertain about their future."
]

# Base Observations for Kit Malthouse
KitMalthouse_observations = [
    "Kit Malthouse is a Conservative MP who has served as the Minister for Crime, Policing and the Fire Service.",
    "During debates, he has stated that the government is committed to ending free movement as part of the Brexit process.",
    "Malthouse has reassured that EU citizens residing in the UK are welcome to stay and that the government has implemented the EU Settlement Scheme to facilitate their continued residence."
]

SteveDouble_observations = [
    "Steve Double is a Conservative MP representing St Austell and Newquay.",
    "He has expressed support for ending free movement, aligning with the government's stance on controlling immigration post-Brexit.",
    "Double has emphasized that the government has provided a clear message that EU citizens currently residing in the UK are welcome to stay, citing the success of the EU Settlement Scheme."
]

TimFarron_observations = [
    "Tim Farron is a Liberal Democrat MP representing Westmorland and Lonsdale.",
    "He has criticized the decision to end free movement, arguing that it sends a negative message to EU citizens and undermines the UK's international standing.",
    "Farron has highlighted the contributions of EU nationals to the UK and has advocated for their rights to be protected post-Brexit."
]

JoStevens_observations = [
    "Jo Stevens is a Labour MP representing Cardiff Central.",
    "She has raised concerns about the impact of ending free movement on universities, noting that it could harm the UK's global reputation and deter international students and academics.",
    "Stevens has advocated for the protection of EU nationals' rights and has questioned the government's approach to immigration post-Brexit."
]

RachaelMaskell_observations = [
    "Rachael Maskell is a Labour/Co-operative MP representing York Central.",
    "She has expressed concerns about the government's immigration policies post-Brexit, particularly regarding family reunification and the rights of unaccompanied minors.",
    "Maskell has questioned the government's preparedness for the consequences of ending free movement and has called for more compassionate immigration policies."
]

In [55]:
# Loop through the observations and add to memory
tuples = [(ChristineJardine_observations, ChristineJardine.memory), 
          (KitMalthouse_observations, KitMalthouse.memory), 
          (SteveDouble_observations, SteveDouble.memory), 
          (TimFarron_observations, TimFarron.memory), 
          (JoStevens_observations, JoStevens.memory), 
          (RachaelMaskell_observations, RachaelMaskell.memory)]

for observations, memory in tuples:
    for observation in observations:
        memory.add_memory(observation)

In [58]:
# View stored memories for each MP
print("Christine Jardine's stored memories:")
print(ChristineJardine_memory.memory_retriever.memory_stream)

print("\nKit Malthouse's stored memories:")
print(KitMalthouse_memory.memory_retriever.memory_stream)

print("\nSteve Double's stored memories:")
print(SteveDouble_memory.memory_retriever.memory_stream)

print("\nTim Farron's stored memories:")
print(TimFarron_memory.memory_retriever.memory_stream)

print("\nJo Stevens's stored memories:")
print(JoStevens_memory.memory_retriever.memory_stream)

print("\nRachael Maskell's stored memories:")
print(RachaelMaskell_memory.memory_retriever.memory_stream)

Christine Jardine's stored memories:
[Document(metadata={'importance': 0.03, 'last_accessed_at': datetime.datetime(2025, 2, 24, 16, 49, 37, 815692), 'created_at': datetime.datetime(2025, 2, 24, 16, 49, 37, 815692), 'buffer_idx': 0}, page_content='Christine Jardine is a Liberal Democrat MP representing Edinburgh West.'), Document(metadata={'importance': 0.045, 'last_accessed_at': datetime.datetime(2025, 2, 24, 16, 49, 38, 397912), 'created_at': datetime.datetime(2025, 2, 24, 16, 49, 38, 397912), 'buffer_idx': 1}, page_content='She has expressed concerns about ending free movement, highlighting its potential negative impact on public services and the economy.'), Document(metadata={'importance': 0.105, 'last_accessed_at': datetime.datetime(2025, 2, 24, 16, 49, 38, 934388), 'created_at': datetime.datetime(2025, 2, 24, 16, 49, 38, 934388), 'buffer_idx': 2}, page_content='Jardine emphasizes the human cost of ending free movement, noting that many EU nationals in her constituency feel unwelco

# Create Simulation

## Input Memory

In [ ]:
import pandas as pd
df_HoC_2000s_raw = pd.read_csv('/kaggle/input/parlspeech/df_HoC_2000s.csv')
df_HoC_2000s_raw.columns

df_HoC_2000s = df_HoC_2000s_raw[['date', 'agenda', 'speechnumber', 'speaker', 'party','text']]

df_HoC_miniDebate = df_HoC_2000s[df_HoC_2000s['agenda'].str.contains('Free Movement of EU Nationals', case=False, na=False)]

del df_HoC_2000s_raw
del df_HoC_2000s

In [66]:
df_HoC_miniDebate#.iloc[3]['text']

,date,agenda,speechnumber,speaker,party,text
1322691,2019-10-02,Free Movement of EU Nationals,426,Christine Jardine,LibDem,"I beg to move, That this House has considered ..."
1322692,2019-10-02,Free Movement of EU Nationals,427,Kit Malthouse,Con,That is not correct.
1322693,2019-10-02,Free Movement of EU Nationals,428,Steve Double,Con,I must take exception to the language used by ...
1322694,2019-10-02,Free Movement of EU Nationals,429,Christine Jardine,LibDem,"With respect, that contrasts completely with w..."
1322695,2019-10-02,Free Movement of EU Nationals,430,Steve Double,Con,It is a fact.
1322696,2019-10-02,Free Movement of EU Nationals,431,Christine Jardine,LibDem,"Although there is a settled status scheme, tha..."
1322697,2019-10-02,Free Movement of EU Nationals,432,Tim Farron,LibDem,Members across the House should understand tha...
1322698,2019-10-02,Free Movement of EU Nationals,433,Christine Jardine,LibDem,My hon. Friend makes an excellent point. I hav...
1322699,2019-10-02,Free Movement of EU Nationals,434,Jo Stevens,Lab,I represent a university constituency and have...
1322700,2019-10-02,Free Movement of EU Nationals,435,Christine Jardine,LibDem,I completely agree with the hon. Lady. There a...


In [57]:
# Process and add observations to each agent's memory
agent_mapping = {
    "Christine Jardine": ChristineJardine, 
    "Kit Malthouse": KitMalthouse, 
    "Steve Double": SteveDouble, 
    "Tim Farron": TimFarron, 
    "Jo Stevens": JoStevens, 
    "Rachael Maskell": RachaelMaskell
}

df_subset = df_HoC_miniDebate.iloc[:3]

for agent in agent_mapping.values():  # Iterate over all agent objects
    for _, row in df_subset.iterrows():  # Iterate over the first three rows
        observation = f"{row['speaker']} said: {row['text']}"
        agent.memory.add_memory(observation)  # Store the observation

## Run Debate
1. Each agent add new-observation into memory. 
2. Each agent does a quick reflection on this new-observation, to whether to "respond or not respond" - depending on personal saliency (a custom function within the class `GenerativeAgent`). Output `decide_to_respond` as either True or False
3. Randomly select one agent from the list of agents that decide to respond to the observation.
4. Print this selected generate_dialogue_response as the new observation.

In [67]:
# List of agents in the debate
agents = [ChristineJardine, KitMalthouse, SteveDouble, TimFarron, JoStevens, RachaelMaskell]
# Define the initial debate topic
initial_observation = 'Christine Jardine responded: "With respect Steve Double, that contrasts completely with what non-UK EU citizens tell me every week on the doorstep."'

In [ ]:
def run_HoC_debate_framework_3 (agents: List[GenerativeAgent],             # get a list of agents
                     initial_observation: str) -> None:         # get the 1st observation
    """Runs a conversation between agents, until a maximum number of turns is reached."""
    
    max_turns = 25
    turns = 0
    
    # Start the debate with an initial observation
    observation = initial_observation
    print(observation)
    
    # Enters a loop where agents take turns generating responses
    while turns < max_turns:
        
        # Step 1: Each agent adds the new observation into memory
        for agent in agents:
            agent.memory.add_memory(observation)
            
        # Step 2: Randomly select one agent from the list of agents that decide to respond to the observation
        responding_agents = [agent for agent in agents if agent.decide_to_respond(observation)]
        if responding_agents:
            agent = random.choice(responding_agents)
            # The selected agent generates a response to the latest observation
            stay_in_dialogue, observation = agent.generate_dialogue_response(observation)
            print(observation)
            
        # Increment the turn count after each full round of responses
        turns += 1

In [68]:
def run_HoC_debate_framework_3(agents: List[GenerativeAgent], 
                               initial_observation: str, 
                               save_path: str) -> None:
    """Runs a conversation between agents and saves the transcript to a file."""
    
    max_turns = 25
    turns = 0
    
    # Start the debate with an initial observation
    observation = initial_observation
    
    # Open the file for writing
    with open(save_path, "w", encoding="utf-8") as file:
        file.write("House of Commons Debate Simulation\n")
        file.write("=" * 50 + "\n")
        file.write(f"Debate Topic: Free Movement of EU Nationals\n")
        file.write("=" * 50 + "\n\n")
        
        # Write the initial observation
        print(observation)
        file.write(f"{observation}\n\n")
    
        # Debate loop
        while turns < max_turns:
            # Step 1: Each agent adds the new observation into memory
            for agent in agents:
                agent.memory.add_memory(observation)
                
            # Step 2: Randomly select an agent who decides to respond
            responding_agents = [agent for agent in agents if agent.decide_to_respond(observation)]
            if responding_agents:
                agent = random.choice(responding_agents)
                
                # Generate response
                stay_in_dialogue, observation = agent.generate_dialogue_response(observation)
                
                # Print response to console
                print(observation)
                
                # Append response to file
                with open(save_path, "a", encoding="utf-8") as file:
                    file.write(f"{observation}\n\n")
            
            # Increment turn count
            turns += 1

In [69]:
debate_output_path = "/kaggle/working/debate_transcript_gpt.txt"

run_HoC_debate_framework_3(agents, initial_observation, debate_output_path)

Christine Jardine responded: "With respect Steve Double, that contrasts completely with what non-UK EU citizens tell me every week on the doorstep."
Christine Jardine said With all due respect, I stand by what non-UK EU citizens in my constituency communicate to me regularly. Their concerns and fears about the end of free movement are real and valid, and it's crucial that we listen to and address them. The human impact of policy decisions cannot be understated, and we must strive to protect the rights and well-being of all individuals, regardless of their nationality. Thank you for bringing attention to this important issue.
Christine Jardine said With all due respect, I appreciate the opportunity to reiterate the concerns and fears that non-UK EU citizens in my constituency have shared with me regarding the end of free movement. It is crucial that we listen to their voices and address their valid concerns to ensure the protection of their rights and well-being. Thank you for bringing 

In [ ]:
run_HoC_debate_framework_3(agents, initial_observation)